# Env

In [ ]:
%%capture
!pip install selenium
!pip install -y chronium-chromedriver
# !pip install chromedriver-autoinstaller


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from IPython.display import clear_output
from tqdm.auto import tqdm
import pandas as pd
import time
import gc

In [ ]:
service = Service()

chrome_options = Options()
# chrome_options.page_load_strategy = 'eager'

chrome_options.page_load_strategy = 'normal'
chrome_options.add_argument("--incognito")

chrome_options.add_argument('--headless')
chrome_options.headless = True
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("disable-infobars")

chrome_options.binary_location='/content/chromedriver.exe'
prefs = {'profile.default_content_setting_values': {'cookies': 2, 'images': 2, 'javascript': 2,
                            'plugins': 2, 'popups': 2, 'geolocation': 2,
                            'notifications': 2, 'auto_select_certificate': 2, 'fullscreen': 2,
                            'mouselock': 2, 'mixed_script': 2, 'media_stream': 2,
                            'media_stream_mic': 2, 'media_stream_camera': 2, 'protocol_handlers': 2,
                            'ppapi_broker': 2, 'automatic_downloads': 2, 'midi_sysex': 2,
                            'push_messaging': 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2,
                            'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement': 2,
                            'durable_storage': 2}}
chrome_options.add_experimental_option('prefs', prefs)

# Oxford

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from lxml import etree
import pandas as pd

thesis=pd.DataFrame({
    'Title':[],'Link':[],'Check':[]
})

page_link=['https://ora.ox.ac.uk/collections/thesis?f%5Bf_type_of_work%5D%5B%5D=Thesis&per_page=100&q=+&search_field=all_fields']

cur_row=len(thesis)
while 1:
    url =  page_link[-1]
    page = urllib.request.urlopen(url)
    htmlparser = etree.HTMLParser()
    tree = etree.parse(page, htmlparser)

    items=tree.xpath('//*[@id="documents"]/div/section/div[2]/h3/a')
    for item in items:
        thesis.loc[cur_row,'Title']=item.text
        thesis.loc[cur_row,'Link']='https://ora.ox.ac.uk/'+item.get('href')
        thesis.loc[cur_row,'Check']='0'
        cur_row=cur_row+1

    next_link=tree.xpath('//*[@id="documents"]/div/div/div/ul/li/a')[-1].get('href')
    if next_link=='#':
        thesis.drop_duplicates(inplace=True)
        thesis.reset_index(drop=True,inplace=True)
        thesis.to_excel('thesis_oxford.xlsx', index=False)
        break
    else:
        print('https://ora.ox.ac.uk/'+next_link)
        page_link=page_link+['https://ora.ox.ac.uk/'+next_link]

In [ ]:
# thesis=pd.read_excel('thesis_oxford.xlsx')
thesis=pd.read_csv('thesis_oxford.csv')

In [ ]:
cnt=1
driver=webdriver.Chrome(service=service,options=chrome_options)
for idx in tqdm(range(len(thesis))):
    if thesis.loc[idx,'Check']=='1' or thesis.loc[idx,'Check']==1:
        continue
    if cnt%50==0:
        driver.close()
        driver=webdriver.Chrome(service=service,options=chrome_options)
    driver.get(thesis.loc[idx,'Link'])
    Expand_abstract=driver.find_elements(By.XPATH,'//div[@class="text-abstract"]//a')
    if len(Expand_abstract)>0:
        Expand_abstract[0].click()

    abstract=driver.find_elements(By.XPATH,'//div[@class="text-abstract"]')
    if len(abstract)==0:
        abstract=driver.find_elements(By.XPATH,'//span[@class="text-abstract"]')
    if len(abstract)>0:
        thesis.loc[idx,'Abstract']=abstract[0].text

    keyword=driver.find_elements(By.XPATH,'//span[@class="text-keyword text-vertical"]//a')
    keyword=[i.get_attribute('innerHTML') for i in keyword]
    thesis.loc[idx,'Keyword']='\n'.join(keyword)


    subject=driver.find_elements(By.XPATH,'//span[@class="text-subject text-vertical"]//a')
    subject=[i.get_attribute('innerHTML') for i in subject]
    thesis.loc[idx,'Subject']='\n'.join(subject)

    advisor=[]
    name=driver.find_elements(By.XPATH,'//span[@class="text-contributors text-vertical"]//a')
    name=[i.text[2:] for i in name]
    metadata=driver.find_elements(By.XPATH,'//div[@class="collapse field-details contributors-details plus-collapse"]')
    metadata=[i.get_attribute('innerHTML') for i in metadata]
    for i,meta in enumerate(metadata):
        if 'Supervisor' in meta and name[i] not in advisor:
            advisor=advisor+[name[i]]
    thesis.loc[idx,'Advisor']='\n'.join(advisor)

    download=driver.find_elements(By.XPATH,'//div[@class="pull-left file-download"]/a')
    download=[i.get_attribute("href") for i in download]
    thesis.loc[idx,'Download']='\n'.join(download)
    thesis.loc[idx,'Check']='1'
    cnt=cnt+1
    if cnt%100==0:
        thesis.to_csv('thesis_oxford.csv', index=False)
        # thesis.to_excel('thesis_oxford.xlsx', index=False)

  0%|          | 0/19435 [00:00<?, ?it/s]

In [ ]:
thesis=pd.read_csv('thesis_oxford.csv')

In [ ]:
mask=thesis['Abstract'].isnull()
len(thesis.loc[mask])
for i,row in thesis.loc[mask].iterrows():
    # print(str(row['Abstract'])=='nan')
    print(row['Link'])
#     if i==10:
#         break

https://ora.ox.ac.uk//objects/uuid:98344a0f-c404-4ed5-9f0c-b10ef196abd2
https://ora.ox.ac.uk//objects/uuid:e8c068d9-994c-417c-8adb-6e57058f8d54
https://ora.ox.ac.uk//objects/uuid:8d73f0dd-c434-4fa5-822f-49f0f344c236
https://ora.ox.ac.uk//objects/uuid:5c642c56-7134-43dd-8859-08af0640d452
https://ora.ox.ac.uk//objects/uuid:b3d6f4fd-1605-41be-b8f5-4cd28ed52983
https://ora.ox.ac.uk//objects/uuid:5471f76d-0d6c-4fc5-8de6-f766650e51fd
https://ora.ox.ac.uk//objects/uuid:f2c9968a-d2c1-4a55-8ead-b79615e69728
https://ora.ox.ac.uk//objects/uuid:78cc4455-02ab-43fd-8485-df9d8a7c8d76
https://ora.ox.ac.uk//objects/uuid:a6332616-6db9-4ba1-973f-6799c231ef12
https://ora.ox.ac.uk//objects/uuid:16d74802-fe7e-4fad-b5c1-d912570c9ab1
https://ora.ox.ac.uk//objects/uuid:0a6a1e6c-3ad5-499d-96bb-cf438620f67f
https://ora.ox.ac.uk//objects/uuid:2d37d17a-3853-401e-9a67-ae2a89be5323
https://ora.ox.ac.uk//objects/uuid:2ec1892f-f77f-4e8a-ac9c-78cda42d95b6
https://ora.ox.ac.uk//objects/uuid:e9c025e3-d63c-4a9d-8584-83e87

In [ ]:
thesis.to_csv('thesis_oxford.csv', index=False)

In [ ]:
thesis.to_csv('thesis_oxford.csv', index=False)
thesis.to_excel('thesis_oxford.xlsx', index=False)

In [ ]:
from google.colab import files
files.download("/content/thesis_oxford.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
driver=webdriver.Chrome(service=service,options=chrome_options)
driver.get('https://ora.ox.ac.uk//objects/uuid:5a8ae151-6261-4ad4-9541-d5c59ba654a7')

In [ ]:
tmp=driver.find_elements(By.XPATH,'//div[@class="text-abstract"]//a')

for i in tmp:
    # print(i.get_attribute('href'))
    print(i.text)

Expand abstract


In [ ]:
tmp=driver.find_elements(By.XPATH,'//*[@id="contributorsDetails0"]//dd')
for i in tmp:
    print(i.get_attribute('innerHTML'))

University of Oxford
MPLS
Physics
Supervisor


# Standford

In [ ]:
# get thesis link
thesis=pd.DataFrame({
    'Title':[], 'Link':[], 'Check':[]
})
page_link=['https://searchworks.stanford.edu/?f%5Baccess_facet%5D%5B%5D=Online&f%5Bbuilding_facet%5D%5B%5D=Stanford+Digital+Repository&f%5Bformat_main_ssim%5D%5B%5D=Book&f%5Bgenre_ssim%5D%5B%5D=Thesis%2FDissertation&per_page=100&search_field=search_author&view=brief']
driver = webdriver.Chrome(service=service,options=chrome_options)
cur_row=0
while(1):
      print(cur_row)
      driver.get(page_link[-1])
      title_list=driver.find_elements(By.XPATH,'//h3[@class="index_title"]//a')
      date_list=driver.find_elements(By.XPATH,'//span[@class="main-title-date"]')
      for idx,title in enumerate(title_list):
          thesis.loc[cur_row,'Title']=title.text
          thesis.loc[cur_row,'Link']=title.get_attribute('href')
          thesis.loc[cur_row,'Publication date']=date_list[idx].text[1:-1]
          thesis.loc[cur_row,'Check']='0'
          cur_row=cur_row+1

      next_page=driver.find_element(By.XPATH,'//a[@aria-label="Go to next page"]').get_attribute('href')
      if next_page[-1]=='#':
          thesis.drop_duplicates(inplace=True)
          thesis.reset_index(drop=True,inplace=True)
          thesis.to_excel('thesis_stanford.xlsx',index=False)
          break
      else:
          page_link.append(next_page)

In [ ]:
thesis=pd.read_excel('thesis_stanford.xlsx')

In [ ]:
def process_contributor(tmp):
    i=0
    author=[]
    advisor=[]
    keyword=[]
    department_school=[]
    while i<len(tmp):
        if tmp[i].tag_name=='dt' and 'Author' in tmp[i].text:
            i=i+1
            while i<len(tmp) and tmp[i].tag_name == 'dd':
                author.append(tmp[i].text)
                # print(tmp[i].text)
                # print(tmp[i].tag_name)
                i=i+1
            i=i-1
        elif tmp[i].tag_name=='dt' and 'Contributor' in tmp[i].text:
            i=i+1
            while i<len(tmp) and tmp[i].tag_name == 'dd':
                if 'dvisor' in tmp[i].text:
                    advisor.append(tmp[i].text)
                else:
                    department_school.append(tmp[i].text)
                i=i+1
            i=i-1
        elif tmp[i].tag_name=='dt' and 'dvisor' in tmp[i].text:
            i=i+1
            while i<len(tmp) and tmp[i].tag_name == 'dd':
                advisor.append(tmp[i].text)
                i=i+1
            i=i-1
        elif tmp[i].tag_name=='dt' and 'Degree granting institution' in tmp[i].text:
            i=i+1
            while i<len(tmp) and tmp[i].tag_name == 'dd':
                temp=tmp[i].text.split(',')
                if len(temp)>1:
                    temp[1].replace('Stanford', "")
                    temp[1].replace('University', "")
                    keyword.append(temp[1])

                i=i+1

            i=i-1
        else:
            i=i+1
            while i<len(tmp) and tmp[i].tag_name == 'dd':
                department_school.append(tmp[i].text)
                i+=i+1
            i=i-1
        i=i+1

    for i in department_school:
        if ' of ' in i:
            idx=i.find(' of ')
            keyword.append(i[idx+4:])
        elif 'Stanford' not in i:
            keyword.append(i)

    for idx,name in enumerate(author):
        temp=name.split(',')
        if len(temp)>1:
            author[idx]=temp[0]+temp[1]
        else:
            author[idx]=temp[0]

    for idx,name in enumerate(advisor):
        temp=name.split(',')
        if len(temp)>1:
            advisor[idx]=temp[0]+temp[1]
        else:
            advisor[idx]=temp[0]
        temp=advisor[idx].split('degree')
        advisor[idx]=temp[0]

    return author,advisor,keyword

In [ ]:
# get thesis detail

cnt=1
driver=webdriver.Chrome(service=service,options=chrome_options)
for idx in tqdm(range(len(thesis))):
    if thesis.loc[idx,'Check']=='1':
        continue
    if cnt%50==0:
        driver.close()
        driver=webdriver.Chrome(service=service,options=chrome_options)
    driver.get(thesis.loc[idx,'Link'])

    contribute_list=driver.find_elements(By.XPATH,'//div[@id="contributors"]/div[@class="section-body"]/dl/*')
    author,advisor,keyword=process_contributor(contribute_list)
    abstract=driver.find_elements(By.XPATH,'//div[@id="abstract-contents"]/div[@class="section-body"]//dd')
    if len(abstract)==0:
        abstract=driver.find_elements(By.XPATH,'//div[@id="contents-summary"]/div[@class="section-body"]//dd')

    subject=driver.find_elements(By.XPATH,'//div[@id="subjects"]/div//dd')
    subject=[i.text for i in subject]
    keyword=keyword+subject

    thesis.loc[idx,'Author'] = '\n'.join(author)
    if len(abstract)>0:
        thesis.loc[idx,'Abstract'] =  abstract[0].text
    thesis.loc[idx,'Advisor'] = '\n'.join(advisor)
    thesis.loc[idx,'Keyword'] = '\n'.join(keyword)
    thesis.loc[idx,'Subject'] = '\n'.join(subject)
    thesis.loc[idx,'Check']='1'
    cnt=cnt+1

    if cnt%100==0:
        thesis.to_csv('thesis_stanford.csv', index=False)
        # thesis.to_excel('thesis_oxford.xlsx', index=False)

In [ ]:
thesis[0:100]

In [ ]:
driver = webdriver.Chrome(service=service,options=chrome_options)

In [ ]:
driver.get('https://searchworks.stanford.edu/view/in00000052165')

In [ ]:
tmp=driver.find_elements(By.XPATH,'//div[@id="contents-summary"]/div[@class="section-body"]//dd')
print(len(tmp))
for i in tmp:
    print(i.text)

1
This dissertation delves into multiple facets of quantum computing and quantum algorithms from the perspective of an applied mathematician or numerical analyst and is segmented into four parts: 1. Efficient and Robust Quantum Phase Estimation (QPE) Algorithms: The first part presents a suite of efficient and robust QPE algorithms, tailored for early fault-tolerant quantum devices in the sense that they (1) use a minimal number of ancilla qubits, (2) allow for inexact initial states with a significant residual, (3) achieve the Heisenberg limit for the total resource used, and (4) have a diminishing prefactor for the maximum circuit length when the residual approaches zero. 2. Advanced Hamiltonian Learning Techniques: The second part discusses two efficient algorithms that deal with the Hamiltonian learning problem for many-body Fermionic and Bosonic systems. 3. Block-encoding of Pseudo-Differential Operators: In the third chapter, the focus shifts to the development of novel algorithm

# UC Berkeley

In [ ]:
thesis=pd.DataFrame({
    'Title':[], 'Link':[], 'Check':[]
})

In [ ]:
# get thesis link
cur_row=len(thesis)
driver = webdriver.Chrome(service=service,options=chrome_options)
for page_num in range(0,9900+1,100):
      if page_num%1500==0:
          driver.close()
          driver = webdriver.Chrome(service=service,options=chrome_options)
      page_link='https://escholarship.org/search?type_of_work=dissertation&pub_year_start=2010&campuses=ucb&sort=desc&rows=100&start='+str(page_num)
      driver.get(page_link)
      while 1:
          try:
              WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'//section[@class="c-scholworks"]//h3//a')))
              item_list=driver.find_elements(By.XPATH,'//section[@class="c-scholworks"]//h3//a')
              for item in item_list:
                  thesis.loc[cur_row,'Link']=item.get_attribute('href')
                  thesis.loc[cur_row,'Check']='0'
                  cur_row=cur_row+1
                  # print(cur_row)
              break
          except Exception as e:
              continue
      print(driver.current_url)

thesis.drop_duplicates(inplace=True)
thesis.reset_index(drop=True,inplace=True)
thesis.to_excel('thesis_berkeley.xlsx',index=False)

In [ ]:
# get thesis detail

cnt=1
driver=webdriver.Chrome(service=service,options=chrome_options)
for idx in tqdm(range(len(thesis))):
    if thesis.loc[idx,'Check']=='1':
        continue
    if cnt%50==0:
        driver.close()
        driver=webdriver.Chrome(service=service,options=chrome_options)

    driver.get(thesis.loc[idx,'Link'])
    title=driver.find_element(By.XPATH,'//h2[@class="c-tabcontent__main-heading"]')
    abstract=driver.find_elements(By.XPATH,'//div[@class="c-clientmarkup"]//p')
    if len(abstract)>0:
        abstract='/n'.join([i.text for i in abstract])
    else:
        abstract=''
    thesis.loc[idx,'Title']=title.text
    thesis.loc[idx,'Abstract']=abstract.text

    driver.get(thesis.loc[idx,'Link']+'#author')
    contributor=driver.find_elements(By.XPATH,'//dl[@class="c-descriptionlist"]')
    author=contributor[0].find_elements(By.XPATH,'.//a')
    author='\n'.join([i.text for i in author])

    advisor=contributor[1].find_elements(By.XPATH,'.//a')
    advisor='\n'.join([i.text for i in advisor])

    thesis.loc[idx,'Author']=author
    thesis.loc[idx,'Advisor']=advisor
    thesis.loc[idx,'Check']='1'
    cnt=cnt+1

    if cnt%100==0:
        thesis.to_excel('thesis_berkeley.xlsx', index=False)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
driver=webdriver.Chrome(service=service,options=chrome_options)
driver.get('https://escholarship.org/uc/item/5861v3dx')

In [ ]:
tmp[50].text

# Toronto

In [ ]:
thesis=pd.DataFrame({
    'Title':[], 'Link':[], 'Check':[]
})


In [ ]:
# get thesis link

driver = webdriver.Chrome(service=service,options=chrome_options)
driver.get('https://tspace.library.utoronto.ca/handle/1807/9945')
def get_thesis_link(link,thesis_type):
    cur_row=len(thesis)
    driver = webdriver.Chrome(service=service,options=chrome_options)
    driver.get(link)
    while 1:
        if cur_row%1000==0:
            clear_output(wait=True)
        print(cur_row)
        print(driver.current_url)
        item_list=driver.find_elements(By.XPATH,'//tbody/tr')[1:]
        for item in item_list:
            title=item.find_element(By.XPATH,'.//td[@headers="t2"]//a')
            thesis.loc[cur_row,'Title']=title.text
            thesis.loc[cur_row,'Link']=title.get_attribute('href')
            thesis.loc[cur_row,'Type']=thesis_type
            thesis.loc[cur_row,'Check']='0'
            cur_row=cur_row+1
            # print(1)

        next=driver.find_elements(By.XPATH,'//div[@style="float:right"]//a[contains(text(),"next")]')
        if len(next)<2:
            break
        else:
            driver.get(next[-1].get_attribute('href'))
    thesis.drop_duplicates(inplace=True)
    thesis.reset_index(drop=True,inplace=True)
    thesis.to_excel('thesis_toronto.xlsx',index=False)
    driver.close()

get_thesis_link('https://tspace.library.utoronto.ca/handle/1807/9945','Doctor thesis')
get_thesis_link('https://tspace.library.utoronto.ca/handle/1807/9947',"Master thesis")
get_thesis_link('https://tspace.library.utoronto.ca/handle/1807/10445',"Thesis")

In [ ]:
# get thesis detail

cnt=1
driver=webdriver.Chrome(service=service,options=chrome_options)
for idx in tqdm(range(len(thesis))):
    if thesis.loc[idx,'Check']=='1':
        continue
    if cnt%50==0:
        driver.close()
        driver=webdriver.Chrome(service=service,options=chrome_options)

    driver.get(thesis.loc[idx,'Link'])

    abstract=driver.find_elements(By.XPATH,'//div[@class="item-field-abstract"]')
    if len(abstract)>0:
        abstract=abstract[0].text
    else:
        abstract=''
    pos=abstract.find('\n')
    thesis.loc[idx,'Abstract']=abstract[pos+1:]

    item_list=driver.find_elements(By.XPATH,'//div[@class="item-field"]')
    for item in item_list:
        item=item.text
        pos=item.find('\n')
        field=item[:pos-1]
        content=item[pos+1:]
        if(field=='Keywords'):
            content=content.split('; ')
            thesis.loc[idx,field] = '\n'.join(content)
        else:
            thesis.loc[idx,field] = content

    thesis.loc[idx,'Check']='1'
    cnt=cnt+1
    if cnt%100==0:
        thesis.to_excel('thesis_toronto.xlsx', index=False)

In [ ]:
driver = webdriver.Chrome(service=service,options=chrome_options)
driver.get('https://tspace.library.utoronto.ca/handle/1807/10445')

In [ ]:
tmp=driver.find_elements(By.XPATH,'//div[@style="float:right"]//a[contains(text(),"next")]')
# tmp=driver.find_elements(By.XPATH,'//div[@style="float:right"]/a')
print(len(tmp))
for i in tmp:
    a=i.text.split('\n')
    print('\n'.join(a[1:]))
    print(i.text)

2

next >

next >


# PostProcess

In [ ]:
oxford=pd.read_csv('thesis_oxford.csv')
stanford=pd.read_csv('thesis_stanford.csv')
berkeley=pd.read_csv('thesis_berkeley.csv')
toronto=pd.read_excel('thesis_toronto.xlsx')
uet=pd.read_excel('thesis_uet.xlsx')

In [ ]:
oxford.rename(columns={'Keyword': 'Keywords'},inplace=True)
stanford.rename(columns={'Keyword': 'Keywords'},inplace=True)
uet.rename(columns={'Keyword': 'Keywords'},inplace=True)

In [ ]:
concate=pd.concat([oxford[['Title','Link','Abstract','Keywords','Advisor']],
                   stanford[['Title','Link','Abstract','Keywords','Advisor']],
                   berkeley[['Title','Link','Abstract','Advisor']],
                   toronto[['Title','Link','Abstract','Keywords','Advisor']]
                  #  uet[['Title','Link','Abstract','Keywords','Advisor']]
                  ],axis=0)
concate.reset_index(drop=True,inplace=True)

In [ ]:
mask=concate['Abstract'].isnull()
concate=concate[~mask]

mask=concate['Abstract'].str.len() >= 100
concate=concate[mask]
# mask=concate['Advisor'].isnull()
# concate=concate[~mask]

concate.drop_duplicates(subset=['Title'])

,Title,Link,Abstract,Keywords,Advisor
0,"Design, fabrication and characterization of me...",https://ora.ox.ac.uk//objects/uuid:7df85434-8b...,This thesis investigates the coupling mechanis...,split ring resonator\nnumerical modeling\nmeta...,"Shamonina, E\nStevens, C"
1,"Thiophene S,S-dioxides as versatile building b...",https://ora.ox.ac.uk//objects/uuid:e421e902-5f...,"In the thesis, the application of diverse thio...",NaN,"Anderson, E"
2,Structural analysis of the cattle antibody rep...,https://ora.ox.ac.uk//objects/uuid:7ffec9c1-88...,"Cattle present an extreme immunological model,...",bRSV\ncrystallography\nbiolayer interferometry...,"Owens, R\nStuart, D\nHammond, J"
3,Measurement of the Higgs boson mass and produc...,https://ora.ox.ac.uk//objects/uuid:029349de-3d...,The ATLAS and CMS experiments at the Large Had...,Higgs physics\ncollider physics\nparticle phys...,"Shipsey, I\nBortoletto, D"
4,"The long poem and its ends, 1797-1869",https://ora.ox.ac.uk//objects/uuid:23ce9f9a-be...,This is a study of five major English poets an...,narrative\nBrowning\nTennyson\nWordsworth\norg...,"Perry, S"
...,...,...,...,...,...
80749,Difference and Distance in Post-Pauline Christ...,https://tspace.library.utoronto.ca/handle/1807...,Three letters near the end of the Pauline corp...,NaN,NaN
80750,When the Valley of the Shadow is Littered with...,https://tspace.library.utoronto.ca/handle/1807...,The author researched the effects of multiple ...,NaN,NaN
80751,"Rediscovering a Critical Theology of Religion,...",https://tspace.library.utoronto.ca/handle/1807...,“Pluralism” is a widely diverse and stimulatin...,NaN,NaN
80752,Women Finding Their Voices in the Small Town M...,https://tspace.library.utoronto.ca/handle/1807...,The concern of this thesis is with the formati...,NaN,NaN


In [ ]:
concate.to_csv('thesis_concat_no_uet_longer_than_100.csv',index=False)

In [ ]:
from google.colab import files
files.download("/content/thesis_concat_no_uet_longer_than_100.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>